In [1]:
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import h5py
import numpy as np
import torch
from bilby.core.prior import Constraint, Cosine, PriorDict, Uniform
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters
from bilby.gw.prior import UniformSourceFrame
from bilby.gw.source import lal_binary_black_hole
from bilby.gw.waveform_generator import WaveformGenerator
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Dark2_8 as palette
from bokeh.plotting import figure
from gwpy.timeseries import TimeSeries
from rich.progress import track

from ml4gw.dataloading import InMemoryDataset
from ml4gw.distributions import Cosine as CosineSampler
from ml4gw.distributions import LogNormal as LogNormalSampler
from ml4gw.distributions import Uniform as UniformSampler
from ml4gw.transforms import RandomWaveformInjection, Whitening
from utils.resnet import ResNet
from utils.tracker import MultiThresholdAUROC, Run

output_notebook()

Loading BokehJS ...

In [2]:
# Paths and what not
BASE_DIR = Path.home() / "bbhnet" / "results"
DATA_DIR = Path.home() / "bbhnet" / "data"
RUN_NAME = "notebook-run"

# Data parameters
START = 1262653854
DURATION = 32768
SAMPLE_RATE = 2048
KERNEL_LENGTH = 2
HIGHPASS = 20

# Injection parameters
WAVEFORM_DURATION = 8
NUM_WAVEFORMS = 50000
REFERENCE_FREQUENCY = 50
MINIMUM_FREQUENCY = 20
INJECTION_FRACTION = 0.5
MEAN_SNR = 15
STD_SNR = 15
MIN_SNR = 1

# Optimization parameters
VALID_FRAC = 0.25
LEARNING_RATE = 4e-3
BATCH_SIZE = 512
PATIENCE = 50
MAX_EPOCHS = 100

In [3]:
background = []
for ifo in "HL":
    ts = TimeSeries.fetch_open_data(
        f"{ifo}1", start=START, end=START + DURATION
    )
    ts = ts.resample(SAMPLE_RATE)
    background.append(ts.value)
background = np.stack(background)

In [4]:
train_length = int((1 - VALID_FRAC) * SAMPLE_RATE * DURATION)
train_background, valid_background = np.split(
    background, [train_length], axis=-1
)

In [5]:
priors = dict(
    mass_1=Uniform(name="mass_1", minimum=5, maximum=100, unit=r"$M_{\odot}$"),
    mass_2=Uniform(name="mass_2", minimum=5, maximum=100, unit=r"$M_{\odot}$"),
    mass_ratio=Constraint(name="mass_ratio", minimum=0.2, maximum=5.0),
    luminosity_distance=UniformSourceFrame(
        name="luminosity_distance", minimum=100, maximum=3000, unit="Mpc"
    ),
    dec=Cosine(name="dec"),
    ra=Uniform(name="ra", minimum=0, maximum=2 * np.pi, boundary="periodic"),
    theta_jn=0,
    psi=0,
    phase=0,
    a_1=0,
    a_2=0,
    tilt_1=0,
    tilt_2=0,
    phi_12=0,
    phi_jl=0,
)
prior_dict = PriorDict(priors)

In [6]:
waveform_generator = WaveformGenerator(
    duration=WAVEFORM_DURATION,
    sampling_frequency=SAMPLE_RATE,
    frequency_domain_source_model=lal_binary_black_hole,
    parameter_conversion=convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={
        "waveform_approximant": "IMRPhenomPv2",
        "reference_frequency": REFERENCE_FREQUENCY,
        "minimum_frequency": MINIMUM_FREQUENCY,
    },
)


def generate_waveform(i):
    row = {k: v[i] for k, v in params.items()}
    polarizations = waveform_generator.time_domain_strain(row)
    polarization_names = sorted(polarizations.keys())
    polarizations = np.stack([polarizations[p] for p in polarization_names])

    # center so that coalescence time is middle sample
    dt = WAVEFORM_DURATION / 2
    polarizations = np.roll(polarizations, int(dt * SAMPLE_RATE), axis=-1)
    return polarizations

10:31 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


The process of actually generating these waveforms can be pretty time consuming, so we'll create a cache file for skipping it on repeated runs. We'll also use multiple threads in case the cache file doesn't exist so that we're not waiting around too long.

In [7]:
WAVEFORMS_FILE = DATA_DIR / "waveforms.h5"
if WAVEFORMS_FILE.exists():
    print("Using local cache file")
    with h5py.File(WAVEFORMS_FILE, "r") as f:
        polarizations = [f[p][:][:, None] for p in ["cross", "plus"]]
        waveforms = np.concatenate(polarizations, axis=1)
        params = {k: v[:] for k, v in f["params"].items()}
else:
    waveforms = np.zeros(
        (NUM_WAVEFORMS, 2, int(SAMPLE_RATE * WAVEFORM_DURATION))
    )
    params = prior_dict.sample(NUM_WAVEFORMS)
    with ThreadPoolExecutor(4) as pool:
        it = pool.map(generate_waveform, range(NUM_WAVEFORMS))
        it = track(it, "Generating waveforms", total=NUM_WAVEFORMS)
        for i, polarizations in enumerate(it):
            waveforms[i] = polarizations

    with h5py.File(WAVEFORMS_FILE, "w") as f:
        f["cross"] = waveforms[:, 0]
        f["plus"] = waveforms[:, 1]
        params_group = f.create_group("params")
        for p, values in params.items():
            params_group[p] = values

Using local cache file


In [8]:
t = np.arange(0, WAVEFORM_DURATION, 1 / SAMPLE_RATE) - WAVEFORM_DURATION / 2
p = figure(
    width=750,
    height=300,
    x_axis_label="Time from coalescence [s]",
    y_axis_label="Gravitational wave strain [unitless]",
    tools="",
)
for i in range(2):
    p.line(
        t,
        waveforms[1, i],
        line_color=palette[i],
        line_alpha=0.8,
        line_width=1.5,
        legend_label=["cross", "plus"][i],
    )
p.legend.click_policy = "hide"
show(p)

In [9]:
num_train = int((1 - VALID_FRAC) * NUM_WAVEFORMS)
train_waveforms, valid_waveforms = np.split(waveforms, [num_train], axis=0)
train_params = {k: v[:num_train] for k, v in params.items()}
valid_params = {k: v[num_train:] for k, v in params.items()}

In [10]:
waveforms_per_batch = int(INJECTION_FRACTION * BATCH_SIZE)
batches_per_epoch = num_train // waveforms_per_batch
train_loader = InMemoryDataset(
    train_background,
    kernel_size=int(KERNEL_LENGTH * SAMPLE_RATE),
    batch_size=BATCH_SIZE,
    coincident=False,
    shuffle=True,
    batches_per_epoch=batches_per_epoch,
)

num_valid_waveforms = int(VALID_FRAC * NUM_WAVEFORMS)
valid_batch_size = 4 * BATCH_SIZE
kernel_size = int(KERNEL_LENGTH * SAMPLE_RATE)
valid_steps = (num_valid_waveforms - 1) // valid_batch_size + 1
valid_loader = InMemoryDataset(
    valid_background,
    kernel_size=kernel_size,
    stride=1,
    batch_size=4 * BATCH_SIZE,
    batches_per_epoch=valid_steps,
    coincident=False,
    shuffle=False,
)

In [11]:
injector = RandomWaveformInjection(
    sample_rate=SAMPLE_RATE,
    ifos=["H1", "L1"],
    dec=CosineSampler(),
    psi=UniformSampler(0, np.pi),
    phi=UniformSampler(-np.pi, np.pi),
    snr=LogNormalSampler(MEAN_SNR, STD_SNR, MIN_SNR),
    highpass=HIGHPASS,
    prob=INJECTION_FRACTION,
    trigger_offset=-0.6,
    cross=train_waveforms[:, 0],
    plus=train_waveforms[:, 1],
)
injector.fit(*train_background, sample_rate=SAMPLE_RATE, fftlength=2)
injector = injector.to("cuda")

valid_injector = RandomWaveformInjection(
    sample_rate=SAMPLE_RATE,
    ifos=["H1", "L1"],
    dec=valid_params["dec"],
    psi=valid_params["psi"],
    phi=valid_params["ra"],
    snr=None,
    highpass=HIGHPASS,
    trigger_offset=0.5,
    cross=valid_waveforms[:, 0],
    plus=valid_waveforms[:, 1],
)
valid_injector = valid_injector.to("cuda")

In [12]:
waveform, sampled_params = injector.sample(1, "cuda")
dec, psi, phi, snr = sampled_params[0]

In [13]:
preprocessor = Whitening(num_channels=2, sample_rate=SAMPLE_RATE, fduration=1)
preprocessor.fit(
    KERNEL_LENGTH, *train_background, fftlength=2, highpass=HIGHPASS
)
preprocessor = preprocessor.to("cuda")

In [14]:
nn = ResNet(2, layers=[2, 2, 2, 2], norm_groups=8).to("cuda")
optimizer = torch.optim.Adam(nn.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-2,
    epochs=MAX_EPOCHS,
    steps_per_epoch=batches_per_epoch,
    anneal_strategy="cos",
)
loss_fn = torch.nn.functional.binary_cross_entropy_with_logits
model = torch.nn.Sequential(preprocessor, nn)

In [15]:
thresholds = [0.01, 0.1, 1]
auroc = MultiThresholdAUROC(thresholds).to("cuda")
run = Run(model, BASE_DIR / RUN_NAME, auroc)

for epoch in run.run(MAX_EPOCHS):
    model.train()
    for X in epoch.track(train_loader, "train_loss", "Training"):
        optimizer.zero_grad(set_to_none=True)
        X = X.to("cuda")

        X, idx, _ = injector(X)
        y_hat = model(X)

        y = torch.zeros((len(X), 1), device="cuda")
        y[idx] = 1

        loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        epoch.update(loss.item())

    model.eval()
    with torch.no_grad():
        waveform_idx = 0
        signal_preds, background_preds = [], []
        for X in epoch.track(valid_loader, "valid_loss", "Validating"):
            X = X.to("cuda")
            y_hat = model(X)
            background_preds.append(y_hat[:, 0])

            y = torch.zeros((len(X), 1), device="cuda")
            background_loss = loss_fn(y_hat, y)

            # this time sample waveforms explicitly
            # and deterministically
            stop = min(waveform_idx + len(X), num_valid_waveforms)
            idx = torch.arange(waveform_idx, stop)
            waveforms, _ = valid_injector.sample(idx, device="cuda")

            X = X[: len(waveforms)]
            y = y[: len(waveforms)] + 1

            # do inference with waveforms near the
            # start, middle, and end of kernels
            buffer = int(0.55 * SAMPLE_RATE)
            middle = waveforms.size(-1) // 2 - buffer
            waveform_loss, waveform_pred = 0, 0
            for i in range(5):
                offset = int(0.9 * i * SAMPLE_RATE / 4)
                start = middle - offset
                stop = start + kernel_size
                X_aug = X + waveforms[:, :, start:stop]
                y_hat = model(X_aug)
                waveform_pred += y_hat[:, 0]
                waveform_loss += loss_fn(y_hat, y)

            waveform_loss /= 5
            waveform_pred /= 5
            signal_preds.append(waveform_pred)

            loss = (background_loss + waveform_loss) / 2
            epoch.update(loss.item())
            waveform_idx += len(X)

        signal_preds = torch.cat(signal_preds)
        background_preds = torch.cat(background_preds)
        aurocs = auroc(signal_preds, background_preds).cpu().numpy()
        epoch.metrics["aurocs"] = {k: v for k, v in zip(thresholds, aurocs)}

run.save()

Output()

In [16]:
model.load_state_dict(torch.load(run.best_weights_path))
t = np.arange(0, 1, 1 / SAMPLE_RATE)
nrows = 6
ncols = 6

X = next(iter(valid_loader))[: nrows * ncols].to("cuda")
idx = torch.arange(nrows * ncols)
waveforms, _ = valid_injector.sample(idx, device="cuda")

start = waveforms.size(-1) // 2 - kernel_size // 2
stop = start + kernel_size
waveforms = waveforms[:, :, start:stop]
X += waveforms
with torch.no_grad():
    X = model._modules["0"](X)
    y = model._modules["1"](X)
X = X.cpu().numpy()
y = y.cpu().numpy()[:, 0]

mass1s = valid_params["mass_1"][: nrows * ncols]
mass2s = valid_params["mass_2"][: nrows * ncols]
dists = valid_params["luminosity_distance"][: nrows * ncols]

rows = []
for i in range(nrows):
    cols = []
    for j in range(ncols):
        k = i * ncols + j
        pred, mass1, mass2, dist = y[k], mass1s[k], mass2s[k], dists[k]
        p = figure(
            title=f"{pred:0.3f}/{mass1:0.1f}/{mass2:0.1f}/{dist:0.1f}",
            x_range=(0, 1),
            y_range=(X.min(), X.max()),
        )
        for ifo in range(2):
            p.line(
                t,
                X[k, ifo],
                line_color=palette[ifo],
                line_width=0.5,
                line_alpha=0.6,
            )
        cols.append(p)
    rows.append(cols)
grid = gridplot(rows, width=225, height=125)
show(grid)